In [2]:
pip install opencv-python 

     |████████████████████████████████| 28.2 MB 1.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import cv2
import numpy as np

st_params = dict(maxCorners=30, qualityLevel=0.2, minDistance=2, blockSize=7)


lk_params  = dict(winSize=(15,15),maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,1))

#video capture
cap = cv2.VideoCapture('Video/run.mp4')


color = (0,255,0)

#Reading and capturing the first frame
ret, first_frame = cap.read()

#convert frames to Grayscale
prev_gray = cv2.cvtColor(first_frame,cv2.COLOR_BGR2GRAY)

#finding the strongest corners in the first frame
prev = cv2.goodFeaturesToTrack(prev_gray,mask=None, **st_params)


#Creating an image with the same dimensions as the frame for later drawing purposes
mask = np.zeros_like(first_frame)

#While Loop
while(cap.isOpened()):
    
    #Read the capture and get the first frame
    ret, frame = cap.read()
    
    #convert all frame to Grayscale(previuosly we did only first frame)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    #calculate optical flow by Lucas-Kanade
    next,status,error=cv2.calcOpticalFlowPyrLK(prev_gray,gray,prev,None,**lk_params)
    
    #select good feature for the previous position
    good_old = prev[status==1]
    
    #select good feature for the next position
    good_new = next[status==1]
    
    #Draw optical flow track
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        
        #return cooridnates for the new point
        a,b = new.ravel()
        
        #return coordinates for the old point
        c,d = old.ravel()
        
        #Draw ine between ew and old position
        mask = cv2.line(mask,
                       (a,b),
                       (c,d),
                       color,
                       2)
    
        
        #draw filled circle
        frame  = cv2.circle(frame,
                           (a,b),
                           3,
                           color,
                           -1)
        
        #overlay optical flow on original frame
        output = cv2.add(frame,mask)
        
        #update the previos frame
        prev_gray = gray.copy()
        
        #update previous good features
        prev = good_new.reshape(-1,1,2)
        
        #open new window and display the output
        cv2.imshow('Optical Flow',output)
        
        #close the frame
        if cv2.waitKey(300)& 0xFF ==ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()
        
        
        



KeyboardInterrupt: 